In [1]:
import numpy as np
import scipy
import scipy.stats
from scipy.stats import qmc
import math
import sys, platform, os
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Loading LHS points
w_lhs, ns_lhs, h_lhs, Ob_lhs, Om_lhs, As_lhs = np.loadtxt('/home/grads/data/Joao/COLA_projects/wCDM/wcdm_lhs.txt', unpack=True)

In [3]:
for n_iter in range(50):
    print(f'Generating .lua file for cosmology {n_iter}')
    Omega_mnu, Omega_c, Omega_b, Omega_m, As, ns, w, h = np.loadtxt(f"cosmo_params/cosmo_{n_iter}.txt", unpack=True)
    
    # COLA SETTINGS
    box_size = 1024
    n_part_1d = 1024
    force_mesh_1d = 2*n_part_1d
    pk_mesh_1d = n_part_1d

    run_name = str(n_iter)
    transferinfo_path = f'/scratch/decola/joao.reboucas2/COLA_projects/wCDM/transferinfo_files/transferinfo{n_iter}.txt'
    output_path = f'/scratch/decola/joao.reboucas2/COLA_projects/wCDM/outputs/{n_iter}'
    lua_out_path = f'/home/grads/data/Joao/COLA_projects/wCDM/lua_files/run_{n_iter}.lua'
    with open(lua_out_path, 'w') as file:
        file.write('all_parameters_must_be_in_file = true\n\n')
        file.write(f'particle_Npart_1D = {n_part_1d}\n')
        file.write(f'force_nmesh = {force_mesh_1d}\n')
        file.write('ic_random_seed = 1234\n')
        file.write('output_redshifts = {3, 2, 1, 0.5, 0.0}\n')
        file.write('timestep_nsteps = {12, 5, 8, 9, 17}\n')
        file.write('output_particles = false\n')
        file.write('fof = false\n')
        file.write('fof_nmin_per_halo = 20\n')
        file.write('pofk = true\n')
        file.write(f'pofk_nmesh = {pk_mesh_1d}\n')
        file.write('ic_fix_amplitude = true\n')
        file.write('ic_reverse_phases = false\n')
        file.write('ic_type_of_input = "transferinfofile"\n')
        file.write('ic_input_filename = "' + transferinfo_path + '"\n\n')
        file.write('output_folder = "' + output_path + '"\n')
        file.write('simulation_name = "run_' + run_name + '"\n')
        file.write('simulation_boxsize = 1024.0\n\n')
        file.write('simulation_use_cola = true\n')
        file.write('simulation_use_scaledependent_cola = true\n\n')
        file.write('cosmology_model = "w0waCDM"\n')
        file.write('cosmology_OmegaCDM = ' + str(Omega_c) + '\n')
        file.write('cosmology_Omegab = ' + str(Omega_b) + '\n')
        file.write('cosmology_OmegaMNu = ' + str(Omega_mnu) + '\n')
        file.write('cosmology_OmegaLambda = 1 - cosmology_OmegaCDM - cosmology_Omegab - cosmology_OmegaMNu\n')
        file.write('cosmology_Neffective = 3.046\n')
        file.write('cosmology_TCMB_kelvin = 2.7255\n')
        file.write('cosmology_h = ' + str(h) + '\n')
        file.write('cosmology_As = ' + str(As) + '\n')
        file.write('cosmology_ns = ' + str(ns) + '\n')
        file.write('cosmology_kpivot_mpc = 0.05\n')
        file.write('if cosmology_model == "w0waCDM" then \n')
        file.write(f'  cosmology_w0 = {w}\n')
        file.write('  cosmology_wa = 0.0\n')
        file.write('end\n\n')
        file.write('if cosmology_model == "DGP" then \n')
        file.write('  cosmology_dgp_OmegaRC = 0.11642\n')
        file.write('end\n\n')
        file.write('gravity_model = "GR"\n\n')
        file.write('if gravity_model == "f(R)" then \n')
        file.write('  gravity_model_fofr_fofr0 = 1e-5\n')
        file.write('  gravity_model_fofr_nfofr = 1.0\n')
        file.write('  gravity_model_screening = true\n')
        file.write('  gravity_model_screening_enforce_largescale_linear = true\n')
        file.write('  gravity_model_screening_linear_scale_hmpc = 0.1\n')
        file.write('end\n\n')
        file.write('if gravity_model == "DGP" then \n')
        file.write('  gravity_model_dgp_rcH0overc = 1.0\n')
        file.write('  gravity_model_screening = true\n')
        file.write('  gravity_model_dgp_smoothing_filter = "tophat"\n')
        file.write('  gravity_model_dgp_smoothing_scale_over_boxsize = 0.0\n')
        file.write('  gravity_model_screening_enforce_largescale_linear = true\n')
        file.write('  gravity_model_screening_linear_scale_hmpc = 0.1\n')
        file.write('end\n\n')
        file.write('particle_allocation_factor = 1.25\n\n')
        file.write('output_fileformat = "GADGET"\n\n')
        file.write('timestep_method = "Quinn"\n')
        file.write('timestep_cola_nLPT = -2.5\n')
        file.write('timestep_algorithm = "KDK"\n')
        file.write('timestep_scalefactor_spacing = "linear"\n')
        file.write('if timestep_scalefactor_spacing == "powerlaw" then\n')
        file.write('  timestep_spacing_power = 1.0\n')
        file.write('end\n\n')
        file.write('ic_random_generator = "GSL"\n')
        file.write('ic_random_field_type = "gaussian"\n')
        file.write('ic_nmesh = particle_Npart_1D\n')
        file.write('ic_use_gravity_model_GR = true\n')
        file.write('ic_LPT_order = 2\n')
        file.write('ic_input_redshift = 19.0\n')
        file.write('ic_initial_redshift = 19.0\n')
        file.write('ic_sigma8_normalization = false\n')
        file.write('ic_sigma8_redshift = 0.0\n')
        file.write('ic_sigma8 = 0.83\n\n')
        file.write('if ic_random_field_type == "nongaussian" then\n')
        file.write('  ic_fnl_type = "local"\n')
        file.write('  ic_fnl = 100.0\n')
        file.write('  ic_fnl_redshift = ic_initial_redshift\n')
        file.write('end\n\n')
        file.write('if ic_random_field_type == "reconstruct_from_particles" then\n')
        file.write('  ic_reconstruct_gadgetfilepath = "output/gadget"\n')
        file.write('  ic_reconstruct_assigment_method = "CIC"\n')
        file.write('  ic_reconstruct_smoothing_filter = "sharpk"\n')
        file.write('  ic_reconstruct_dimless_smoothing_scale = 1.0 / (128 * math.pi)\n')
        file.write('  ic_reconstruct_interlacing = false\n')
        file.write('end\n\n')
        file.write('if ic_random_field_type == "read_particles" then\n')
        file.write('  ic_reconstruct_gadgetfilepath = "path/gadget"\n')
        file.write('end\n\n')
        file.write('force_density_assignment_method = "CIC"\n')
        file.write('force_kernel = "continuous_greens_function"\n')
        file.write('force_linear_massive_neutrinos = true\n\n')
        file.write('fof_linking_length = 0.2 / particle_Npart_1D\n')
        file.write('fof_nmesh_max = 0\n\n')
        file.write('pofk_interlacing = true\n')
        file.write('pofk_subtract_shotnoise = false\n')
        file.write('pofk_density_assignment_method = "PCS"\n\n')
        file.write('pofk_multipole = false\n')
        file.write('pofk_multipole_nmesh = 128\n')
        file.write('pofk_multipole_interlacing = true\n')
        file.write('pofk_multipole_subtract_shotnoise = false\n')
        file.write('pofk_multipole_ellmax = 4\n')
        file.write('pofk_multipole_density_assignment_method = "PCS"\n\n')
        file.write('bispectrum = false\n')
        file.write('bispectrum_nmesh = 128\n')
        file.write('bispectrum_nbins = 10\n')
        file.write('bispectrum_interlacing = true\n')
        file.write('bispectrum_subtract_shotnoise = false\n')
        file.write('bispectrum_density_assignment_method = "PCS"\n')

    print(f'Saved .lua file for cosmology {n_iter} in {lua_out_path}.')

Generating .lua file for cosmology 0
Saved .lua file for cosmology 0 in /home/grads/data/Joao/COLA_projects/wCDM/lua_files/run_0.lua.
Generating .lua file for cosmology 1
Saved .lua file for cosmology 1 in /home/grads/data/Joao/COLA_projects/wCDM/lua_files/run_1.lua.
Generating .lua file for cosmology 2
Saved .lua file for cosmology 2 in /home/grads/data/Joao/COLA_projects/wCDM/lua_files/run_2.lua.
Generating .lua file for cosmology 3
Saved .lua file for cosmology 3 in /home/grads/data/Joao/COLA_projects/wCDM/lua_files/run_3.lua.
Generating .lua file for cosmology 4
Saved .lua file for cosmology 4 in /home/grads/data/Joao/COLA_projects/wCDM/lua_files/run_4.lua.
Generating .lua file for cosmology 5
Saved .lua file for cosmology 5 in /home/grads/data/Joao/COLA_projects/wCDM/lua_files/run_5.lua.
Generating .lua file for cosmology 6
Saved .lua file for cosmology 6 in /home/grads/data/Joao/COLA_projects/wCDM/lua_files/run_6.lua.
Generating .lua file for cosmology 7
Saved .lua file for cosmo

In [4]:
# Test lua inputs
for n_iter, w in enumerate([-1.3, -0.7, -0.999]):
    run_name = f"test_{n_iter}"
    # Reading cosmo parameters from CLASS
    Omega_mnu, Omega_c, Omega_b, Omega_m, As, ns, w, h = np.loadtxt(f"cosmo_params/cosmo_{run_name}.txt", unpack=True)
    
    # COLA SETTINGS
    box_size = 1024
    n_part_1d = 1024
    force_mesh_1d = 2*n_part_1d
    pk_mesh_1d = n_part_1d
    print(f"Settings: L = {box_size}, Np1d = {n_part_1d}, Force_NMesh = {force_mesh_1d}, pk_mesh = {pk_mesh_1d}")

    
    transferinfo_path = f'/gpfs/projects/MirandaGroup/jonathan/cola_projects/wcdm/transferinfo_files/transferinfo{n_iter}.txt'
    output_path = f'/gpfs/projects/MirandaGroup/jonathan/cola_projects/wcdm/output/{run_name}'
    lua_out_path = f'/home/grads/data/Joao/COLA_projects/wCDM/lua_files/{run_name}.lua'
    with open(lua_out_path, 'w') as file:
        file.write('all_parameters_must_be_in_file = true\n\n')
        file.write(f'particle_Npart_1D = {n_part_1d}\n')
        file.write(f'force_nmesh = {force_mesh_1d}\n')
        file.write('ic_random_seed = 1234\n')
        file.write('output_redshifts = {3, 2, 1, 0.5, 0.0}\n')
        file.write('timestep_nsteps = {12, 5, 8, 9, 17}\n')
        file.write('output_particles = false\n')
        file.write('fof = false\n')
        file.write('fof_nmin_per_halo = 20\n')
        file.write('pofk = true\n')
        file.write(f'pofk_nmesh = {pk_mesh_1d}\n')
        file.write('ic_fix_amplitude = true\n')
        file.write('ic_reverse_phases = false\n')
        file.write('ic_type_of_input = "transferinfofile"\n')
        file.write('ic_input_filename = "' + transferinfo_path + '"\n\n')
        file.write('output_folder = "' + output_path + '"\n')
        file.write('simulation_name = "run_' + run_name + '"\n')
        file.write(f'simulation_boxsize = {box_size:.1f}\n\n')
        file.write('simulation_use_cola = true\n')
        file.write('simulation_use_scaledependent_cola = true\n\n')
        file.write('cosmology_model = "w0waCDM"\n')
        file.write('cosmology_OmegaCDM = ' + str(Omega_c) + '\n')
        file.write('cosmology_Omegab = ' + str(Omega_b) + '\n')
        file.write('cosmology_OmegaMNu = ' + str(Omega_mnu) + '\n')
        file.write('cosmology_OmegaLambda = 1 - cosmology_OmegaCDM - cosmology_Omegab - cosmology_OmegaMNu\n')
        file.write('cosmology_Neffective = 3.046\n')
        file.write('cosmology_TCMB_kelvin = 2.7255\n')
        file.write('cosmology_h = ' + str(h) + '\n')
        file.write('cosmology_As = ' + str(As) + '\n')
        file.write('cosmology_ns = ' + str(ns) + '\n')
        file.write('cosmology_kpivot_mpc = 0.05\n')
        file.write('if cosmology_model == "w0waCDM" then \n')
        file.write(f'  cosmology_w0 = {w}\n')
        file.write('  cosmology_wa = 0.0\n')
        file.write('end\n\n')
        file.write('if cosmology_model == "DGP" then \n')
        file.write('  cosmology_dgp_OmegaRC = 0.11642\n')
        file.write('end\n\n')
        file.write('gravity_model = "GR"\n\n')
        file.write('if gravity_model == "f(R)" then \n')
        file.write('  gravity_model_fofr_fofr0 = 1e-5\n')
        file.write('  gravity_model_fofr_nfofr = 1.0\n')
        file.write('  gravity_model_screening = true\n')
        file.write('  gravity_model_screening_enforce_largescale_linear = true\n')
        file.write('  gravity_model_screening_linear_scale_hmpc = 0.1\n')
        file.write('end\n\n')
        file.write('if gravity_model == "DGP" then \n')
        file.write('  gravity_model_dgp_rcH0overc = 1.0\n')
        file.write('  gravity_model_screening = true\n')
        file.write('  gravity_model_dgp_smoothing_filter = "tophat"\n')
        file.write('  gravity_model_dgp_smoothing_scale_over_boxsize = 0.0\n')
        file.write('  gravity_model_screening_enforce_largescale_linear = true\n')
        file.write('  gravity_model_screening_linear_scale_hmpc = 0.1\n')
        file.write('end\n\n')
        file.write('particle_allocation_factor = 1.25\n\n')
        file.write('output_fileformat = "GADGET"\n\n')
        file.write('timestep_method = "Quinn"\n')
        file.write('timestep_cola_nLPT = -2.5\n')
        file.write('timestep_algorithm = "KDK"\n')
        file.write('timestep_scalefactor_spacing = "linear"\n')
        file.write('if timestep_scalefactor_spacing == "powerlaw" then\n')
        file.write('  timestep_spacing_power = 1.0\n')
        file.write('end\n\n')
        file.write('ic_random_generator = "GSL"\n')
        file.write('ic_random_field_type = "gaussian"\n')
        file.write('ic_nmesh = particle_Npart_1D\n')
        file.write('ic_use_gravity_model_GR = true\n')
        file.write('ic_LPT_order = 2\n')
        file.write('ic_input_redshift = 19.0\n')
        file.write('ic_initial_redshift = 19.0\n')
        file.write('ic_sigma8_normalization = false\n')
        file.write('ic_sigma8_redshift = 0.0\n')
        file.write('ic_sigma8 = 0.83\n\n')
        file.write('if ic_random_field_type == "nongaussian" then\n')
        file.write('  ic_fnl_type = "local"\n')
        file.write('  ic_fnl = 100.0\n')
        file.write('  ic_fnl_redshift = ic_initial_redshift\n')
        file.write('end\n\n')
        file.write('if ic_random_field_type == "reconstruct_from_particles" then\n')
        file.write('  ic_reconstruct_gadgetfilepath = "output/gadget"\n')
        file.write('  ic_reconstruct_assigment_method = "CIC"\n')
        file.write('  ic_reconstruct_smoothing_filter = "sharpk"\n')
        file.write('  ic_reconstruct_dimless_smoothing_scale = 1.0 / (128 * math.pi)\n')
        file.write('  ic_reconstruct_interlacing = false\n')
        file.write('end\n\n')
        file.write('if ic_random_field_type == "read_particles" then\n')
        file.write('  ic_reconstruct_gadgetfilepath = "path/gadget"\n')
        file.write('end\n\n')
        file.write('force_density_assignment_method = "CIC"\n')
        file.write('force_kernel = "continuous_greens_function"\n')
        file.write('force_linear_massive_neutrinos = true\n\n')
        file.write('fof_linking_length = 0.2 / particle_Npart_1D\n')
        file.write('fof_nmesh_max = 0\n\n')
        file.write('pofk_interlacing = true\n')
        file.write('pofk_subtract_shotnoise = false\n')
        file.write('pofk_density_assignment_method = "PCS"\n\n')
        file.write('pofk_multipole = false\n')
        file.write('pofk_multipole_nmesh = 128\n')
        file.write('pofk_multipole_interlacing = true\n')
        file.write('pofk_multipole_subtract_shotnoise = false\n')
        file.write('pofk_multipole_ellmax = 4\n')
        file.write('pofk_multipole_density_assignment_method = "PCS"\n\n')
        file.write('bispectrum = false\n')
        file.write('bispectrum_nmesh = 128\n')
        file.write('bispectrum_nbins = 10\n')
        file.write('bispectrum_interlacing = true\n')
        file.write('bispectrum_subtract_shotnoise = false\n')
        file.write('bispectrum_density_assignment_method = "PCS"\n')

Settings: L = 1024, Np1d = 1024, Force_NMesh = 2048, pk_mesh = 1024
Settings: L = 1024, Np1d = 1024, Force_NMesh = 2048, pk_mesh = 1024
Settings: L = 1024, Np1d = 1024, Force_NMesh = 2048, pk_mesh = 1024


In [10]:
Omega_mnu, Omega_c, Omega_b, Omega_m, As, ns, h = np.loadtxt(f"cosmo_params/cosmo_test_0.txt", unpack=True)